PLEASE FILL IN SECTIONS BELOW, REPLACING ITALIC TEXT WITH YOUR CONTENT

# Introduction
The data source is Census data that quantifies several attributes of commute practices in Orange County of workers aged 16 and up. The data contains several categories, for example:

- Mode of transportation (Car, Van, Truck, Bicycle, Taxicab and motorcycle)
- Average time to work
- Number of vehicles contained in the household (1, 2, or 3)
- Data is also by day

## Questions to answer

- What is the average commute time to work within Orange County
- What is the average number of vehicles per household in Orange County
- 

# Setup
## Data
VERY, VERY IMPORTANT PLEASE PROVIDE DEATILS ABOUT THE DATA YOU USED AND HOW TO GET IT SO OTHERS CAN RUN THIS AMAZING NOTEBOOK TOO

|Dataset               | Type           | Description                                     |  Where to get the data |
|----------------------|----------------|-------------------------------------------------|------------------------|
|                      |                |                                                 |                        |
|                      |                |                                                 |                        |
|                      |                |                                                 |                        |

## Environment
PLEASE RUN THE PANEL BELOW SO OTHERS KNOW EXACTLY THE ENVIRONMENT

In [1]:
# Add package installs here, for example ...
# !pip install pycurl

#!python -V
#!pip list

# list of package dependencies and their pip install commands
#!pip install geopandas
#!pip install census
#!pip install us
#!pip install geopandas hvplot


In [23]:
# Add imports here
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')
import os
import re
from census import Census
from us import states
from pathlib import Path

     

## Settings
PLEASE USE VARIABLES FOR THINGS LIKE FILE LOCATIONS AND CONFIGURATION SETTINGS

In [3]:
# Some examples ...
#DATA_DIRECTORY = './/'
#USER_NAME = ''

# Please always use environment variables for sensative credentials, never commit credentials to github
#USER_PASSWORD = os.environ['USER_PASSWORD']

## Data

In [4]:
# Put code here for getting your data. Use tools such as curl and wget if data is available online
# used data provided from US census. Downloaded csv file to local machine. 
# input csv file and put into dataframe

csvpath = Path("ACSST5Y2021.S0801-Data.csv")

#read in csv file and set to dataframe
commute_df = pd.read_csv(csvpath)

#show dataframe
commute_df.head()


,GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_001MA,S0801_C01_001EA,S0801_C01_002E,S0801_C01_002M,S0801_C01_002MA,S0801_C01_002EA,...,S0801_C03_055EA,S0801_C03_056E,S0801_C03_056M,S0801_C03_056MA,S0801_C03_056EA,S0801_C03_057E,S0801_C03_057M,S0801_C03_057MA,S0801_C03_057EA,Unnamed: 686
0,Geography,Geographic Area Name,Estimate!!Total!!Workers 16 years and over,Margin of Error!!Total!!Workers 16 years and over,Annotation of Margin of Error!!Total!!Workers ...,Annotation of Estimate!!Total!!Workers 16 year...,Estimate!!Total!!Workers 16 years and over!!ME...,Margin of Error!!Total!!Workers 16 years and o...,Annotation of Margin of Error!!Total!!Workers ...,Annotation of Estimate!!Total!!Workers 16 year...,...,Annotation of Estimate!!Female!!PERCENT ALLOCA...,Estimate!!Female!!PERCENT ALLOCATED!!Travel ti...,Margin of Error!!Female!!PERCENT ALLOCATED!!Tr...,Annotation of Margin of Error!!Female!!PERCENT...,Annotation of Estimate!!Female!!PERCENT ALLOCA...,Estimate!!Female!!PERCENT ALLOCATED!!Vehicles ...,Margin of Error!!Female!!PERCENT ALLOCATED!!Ve...,Annotation of Margin of Error!!Female!!PERCENT...,Annotation of Estimate!!Female!!PERCENT ALLOCA...,NaN
1,1400000US12095010201,"Census Tract 102.01, Orange County, Florida",1315,305,NaN,NaN,73.2,10.1,NaN,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
2,1400000US12095010202,"Census Tract 102.02, Orange County, Florida",2441,459,NaN,NaN,70.5,7.5,NaN,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
3,1400000US12095010300,"Census Tract 103, Orange County, Florida",1515,279,NaN,NaN,82.1,7.3,NaN,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
4,1400000US12095010400,"Census Tract 104, Orange County, Florida",333,80,NaN,NaN,71.8,15.2,NaN,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN


In [5]:
# check the number of rows and columns in the dataframe
commute_df.shape

(268, 687)

In [6]:
# check the variable type in the dataframe
commute_df.dtypes

GEO_ID              object
NAME                object
S0801_C01_001E      object
S0801_C01_001M      object
S0801_C01_001MA     object
                    ...   
S0801_C03_057E      object
S0801_C03_057M      object
S0801_C03_057MA     object
S0801_C03_057EA     object
Unnamed: 686       float64
Length: 687, dtype: object

## Data Cleaning
- The metadata file has a description of the values in each column in the csv file. I used this file as a key to understand what data is contained within the columns of the csv file. The data included in the file shows the time it takes to commute within Orange County, as well as the number of available to workers 16 and up. I found that the following columns helpful:
- S0801_C01_046E = The travel time to work in minutes for workers 16yrs and older
- S0801_C02_046E = The travel time to work in minutes for male workers 16 yrs and older
- S0801_C03_046E = The travel time to work in minutes for female workers 16 yrs an older
- S0801_C01_047E = The number of vehicles available within the census tract
- S0801_C01_048E = The number of households with no vehicle available
- S0801_C01_049E = The number of households with 1 vehicle available
- S0801_C01_050E = The number of households with 2 vehicles available
- S0801_C01_051E = The number of households with 3 vehicles available


In [7]:
#remove columns with NaN values
commute_df_clean = commute_df.dropna(axis = 'columns')

commute_df_clean.shape


(268, 374)

In [8]:
# select columns that correspond to data on average travel time to work by sex
# data can be summarized to calculate the overall average commute times within orange county

avg_time_df = commute_df_clean.loc[ :,['GEO_ID', 'NAME','S0801_C01_046E','S0801_C02_046E','S0801_C03_046E']]

# view dataframe to validate dataframe slice
pd.concat([avg_time_df.head(), avg_time_df.tail()])

,GEO_ID,NAME,S0801_C01_046E,S0801_C02_046E,S0801_C03_046E
0,Geography,Geographic Area Name,Estimate!!Total!!Workers 16 years and over who...,Estimate!!Male!!Workers 16 years and over who ...,Estimate!!Female!!Workers 16 years and over wh...
1,1400000US12095010201,"Census Tract 102.01, Orange County, Florida",22.1,26.0,18.5
2,1400000US12095010202,"Census Tract 102.02, Orange County, Florida",24.8,20.6,30.9
3,1400000US12095010300,"Census Tract 103, Orange County, Florida",20.7,22.1,19.1
4,1400000US12095010400,"Census Tract 104, Orange County, Florida",44.6,58.1,27.3
263,1400000US12095018800,"Census Tract 188, Orange County, Florida",24.5,24.7,24.3
264,1400000US12095018901,"Census Tract 189.01, Orange County, Florida",17.0,18.2,16.0
265,1400000US12095018902,"Census Tract 189.02, Orange County, Florida",29.0,31.1,26.6
266,1400000US12095019000,"Census Tract 190, Orange County, Florida",22.7,25.0,20.6
267,1400000US12095990000,"Census Tract 9900, Orange County, Florida",-,-,-


In [9]:
# selecting the columns from the dataframe that correspond to commute time in minutes
# convert the commute times in minutes from an python object to a numerical value

numbr_cols = avg_time_df[['S0801_C01_046E','S0801_C02_046E','S0801_C03_046E']]

#converting values from string to float
for cols in numbr_cols:
    try:
        avg_time_df[cols]= pd.to_numeric(avg_time_df[cols],errors = 'coerce')
    except ValueError:
        pass 

avg_time_df.dtypes


GEO_ID             object
NAME               object
S0801_C01_046E    float64
S0801_C02_046E    float64
S0801_C03_046E    float64
dtype: object

In [18]:
# renaming the dataframe columns to meaningful names 
avg_time_df = avg_time_df.rename(columns ={'S0801_C01_046E':'Total_Workers_Commute_Time','S0801_C02_046E':'Male_Workers_Commute_Time',
                                           'S0801_C03_046E' :'Female_Workers_Commute_Time'})
avg_time_df.head()

,GEO_ID,NAME,Total_Workers_Commute_Time,Male_Workers_Commute_Time,Female_Workers_Commute_Time
0,Geography,Geographic Area Name,NaN,NaN,NaN
1,1400000US12095010201,"Census Tract 102.01, Orange County, Florida",22.1,26.0,18.5
2,1400000US12095010202,"Census Tract 102.02, Orange County, Florida",24.8,20.6,30.9
3,1400000US12095010300,"Census Tract 103, Orange County, Florida",20.7,22.1,19.1
4,1400000US12095010400,"Census Tract 104, Orange County, Florida",44.6,58.1,27.3


In [19]:
# remove the first row from the dataframe
avg_time_df.drop(0)

,GEO_ID,NAME,Total_Workers_Commute_Time,Male_Workers_Commute_Time,Female_Workers_Commute_Time
1,1400000US12095010201,"Census Tract 102.01, Orange County, Florida",22.1,26.0,18.5
2,1400000US12095010202,"Census Tract 102.02, Orange County, Florida",24.8,20.6,30.9
3,1400000US12095010300,"Census Tract 103, Orange County, Florida",20.7,22.1,19.1
4,1400000US12095010400,"Census Tract 104, Orange County, Florida",44.6,58.1,27.3
5,1400000US12095010500,"Census Tract 105, Orange County, Florida",38.1,24.6,44.8
...,...,...,...,...,...
263,1400000US12095018800,"Census Tract 188, Orange County, Florida",24.5,24.7,24.3
264,1400000US12095018901,"Census Tract 189.01, Orange County, Florida",17.0,18.2,16.0
265,1400000US12095018902,"Census Tract 189.02, Orange County, Florida",29.0,31.1,26.6
266,1400000US12095019000,"Census Tract 190, Orange County, Florida",22.7,25.0,20.6


In [20]:
# drop all of the na values in the dataframe
avg_time_df.dropna()

,GEO_ID,NAME,Total_Workers_Commute_Time,Male_Workers_Commute_Time,Female_Workers_Commute_Time
1,1400000US12095010201,"Census Tract 102.01, Orange County, Florida",22.1,26.0,18.5
2,1400000US12095010202,"Census Tract 102.02, Orange County, Florida",24.8,20.6,30.9
3,1400000US12095010300,"Census Tract 103, Orange County, Florida",20.7,22.1,19.1
4,1400000US12095010400,"Census Tract 104, Orange County, Florida",44.6,58.1,27.3
5,1400000US12095010500,"Census Tract 105, Orange County, Florida",38.1,24.6,44.8
...,...,...,...,...,...
262,1400000US12095018700,"Census Tract 187, Orange County, Florida",27.5,24.3,30.0
263,1400000US12095018800,"Census Tract 188, Orange County, Florida",24.5,24.7,24.3
264,1400000US12095018901,"Census Tract 189.01, Orange County, Florida",17.0,18.2,16.0
265,1400000US12095018902,"Census Tract 189.02, Orange County, Florida",29.0,31.1,26.6


## Analysis

In [22]:
#summary statistics
avg_time_df.describe()

,Total_Workers_Commute_Time,Male_Workers_Commute_Time,Female_Workers_Commute_Time
count,266.000000,265.000000,266.000000
mean,28.375564,29.520377,27.096241
std,5.123881,6.171946,6.003928
min,13.900000,13.500000,13.100000
25%,25.000000,25.600000,23.300000
50%,28.050000,28.900000,26.700000
75%,31.000000,32.300000,29.800000
max,53.100000,59.700000,69.000000


### Average commute time within Orange County for workers 16 and up is around 28.37 minutes, with a max time of 53 minutes. 